In [1]:
import sys
sys.path.append('./Softs/')
import Spanve_beta as Spanve
print(Spanve.__version__)
from Spanve_beta import *
import os
import scanpy as sc
save_dir = './recode/IMPUTEDLIBD/data/'
data_dirs = './data/SpatialLIBD/'
data_ids = [file for file in os.listdir(data_dirs) if file.endswith('.h5ad')]

0.1.4_beta


In [2]:
from collections import defaultdict
import os
import pandas as pd
import numpy as np
import scanpy as sc
from statsmodels.stats.multitest import multipletests

softs = [
    'spanve-k','spanve-d',
    'somde','moran','geary',
    'r-sparkx','r-gitto-rank','sepal',
    'r-gitto-sirank','spatialde','p.spanve-k','p.spanve-d', 'cell_ranger'
]
    
index_cols = defaultdict(int)
index_cols.update({
    "r-gitto-rank":1,
    'r-gitto-sirank':1,
    'somde':3,
    'spatialde':3,
})

p_use_columns = {
    "geary":"pval_norm_fdr_bh",
    "moran":"pval_norm_fdr_bh",
    "r-gitto-rank":"adj.p.value",
    "r-sparkx":"adjustedPval",
    'somde':'qval',
    'spatialde':'qval',
    'spanve-d':'fdrs',
    'spanve-k':'fdrs',
    'p.spanve-d':'fdrs',
    'p.spanve-k':'fdrs'
}
score_use_columns = {
    'r-gitto-sirank':'scores',
    'sepal':'average',
    'cell_ranger':'dispersions_norm'
}


result_dir = "./recode/SpatialLIBD/"
# data_ids = os.listdir(result_dir)

def parse_file(data_dir,soft):
    path = os.path.join(result_dir,data_dir,soft)
    try:
        file = [i for i in os.listdir(path) if i.endswith('csv')][0]
    except:
        print(data_dir,soft,'have no result file.')
        return None
    file = os.path.join(path,file)
    if soft not in p_use_columns:
        use_col = score_use_columns[soft]
    elif soft in p_use_columns:
        use_col = p_use_columns[soft]
    else: print(soft)
        
    df = pd.read_csv(file,index_col=index_cols[soft])
    if soft in ['moran','geary']:
        _0, fdrs, _1, _2, = multipletests(df['pval_norm'].fillna(1), method='fdr_bh')
        df[use_col] = fdrs
    df = df[use_col]
    df.index.name = 'genes'
    df = df.reset_index().groupby(by='genes').mean().iloc[:,0]
    df.name = soft
    return df

In [3]:
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

get_genes_thres = lambda s: s.index[s < 0.05]

bar = tqdm(total=len(data_ids))

for data_id in data_ids:
    bar.set_description_str(f'read and process data @{data_id}')
    
    adata = sc.read_h5ad(os.path.join(data_dirs,data_id))
    # pre_adata = adata_preprocess_int(adata)
    # row_counts = adata.X.copy()
    adata.layers['counts'] = adata.X.copy()
    # pre_X = adata_preprocess(adata).X
    sc.pp.normalize_total(adata)
    sc.pp.log1p(adata)
    
    bar.set_description_str(f'running other methods and impute(pca)')
    sc.pp.highly_variable_genes(adata, flavor='cell_ranger')
    # sc.pp.scale(adata)
    
    sc.pp.pca(adata, use_highly_variable=True)
    adata.obsm['cell_ranger.pca'] = adata.obsm['X_pca'].copy()
    
    genes_dict = dict()
    for method in ['moran','spatialde','spanve-k','p.spanve-k']:
        bar.set_description_str(f'running pca impatation at {method}')
        genes_dict[method] = get_genes_thres(parse_file(data_id,method))
        adata.obsm[f'{method}.pca'] = sc.pp.pca(adata[:, genes_dict[method]].X)
    
    # spanve imputation
    pre_X = adata.X.copy()
    del adata.uns['log1p']
    adata.X = adata.layers['counts'].copy()
    model = Spanve(adata,verbose=False)
    model.finding_spatial_neibors(model.K)
    model.nodist_hypoth(model.X)
    for method in ['spanve-k','p.spanve-k']:
        bar.set_description_str(f'running spanve impatation at {method}')
        psedo_reject = adata.var_names.isin(genes_dict[method])
        graph = model.build_sample_graph_from_svgenes(select = psedo_reject)
        Xi = model.impute_from_graph(X = pre_X[:,psedo_reject], graph = graph)
        adata.obsm[f'{method}.imputated.pca'] = sc.pp.pca(Xi)

    adata.var = adata.var.fillna(False)
    adata.write_h5ad(os.path.join(save_dir, data_id))
    bar.update(1)
    
bar.close()

running spanve impatation at p.spanve-k: 100%|██████████| 12/12 [04:22<00:00, 21.91s/it]
